In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from typing import Tuple, List, Any, Union, Optional
from joblib import Parallel, delayed

In [11]:
# Read the reference sequence from chrX.fa
# >chrX
# line
# line

def readFasta(file) -> str:
    lines = np.loadtxt(file, dtype=str)
    # Remove the first line
    lines = lines[1:]
    seq = ''.join(lines)
    return seq


In [12]:
# Get the last column of the BWT by reading the chrX_last_col.txt file
# this last column will have one extra character which is the $ sign
def readFastcol(file) -> str:
    lines = np.loadtxt(file, dtype=str)
    lastCol = ''.join(lines)
    return lastCol


In [13]:
# Get the first column of the BWT by sorting the last column
def getFirstcol(last_col) -> str:
    firstCol = ''.join(sorted(last_col))
    firstCol = firstCol.replace('$', '') + '$'
    #Give the index of the last A
    print(firstCol.index('A'))
    return firstCol

In [14]:
# Get the list of reads from the reads file
def getReads(file: str) -> np.ndarray[str]:
    reads = np.loadtxt(file, dtype=str)
    return reads

In [15]:
#Get the mapping of the last column characters to the reference sequence
def getMapping(file) -> np.ndarray[int]:
    lines = np.loadtxt(file, dtype=int)
    return lines

In [62]:
#initialize 4 arrays for milestones of the four characters A, C, G, T - these will be used by the rank query
AMilestones = []
CMilestones = []
GMilestones = []
TMilestones = []
controlA = controlC = controlG = controlT = 0
milestone = 1000

In [7]:
reference = readFasta('chrX.fa')

KeyboardInterrupt: 

In [64]:
print('Reference sequence length:', len(reference))

Reference sequence length: 151100560


In [156]:
reference[0]

'C'

In [49]:
def createMilestones(BWT, milestone) -> Tuple[np.ndarray[int], np.ndarray[int], np.ndarray[int], np.ndarray[int]]:
    AMilestones = []
    CMilestones = []
    GMilestones = []
    TMilestones = []
    controlA = controlC = controlG = controlT = 0

    for element in range(len(BWT)):
        if BWT[element] == 'A':
            controlA += 1
        elif BWT[element] == 'C':
            controlC += 1
        elif BWT[element] == 'G':
            controlG += 1
        elif BWT[element] == 'T':
            controlT += 1
        else:
            pass

        if element % milestone == 0:
            AMilestones.append(controlA)
            CMilestones.append(controlC)
            GMilestones.append(controlG)
            TMilestones.append(controlT)
        else:
            pass

    AMilestones.append(controlA)
    CMilestones.append(controlC)
    GMilestones.append(controlG)
    TMilestones.append(controlT)

    AMilestones = np.array(AMilestones)
    CMilestones = np.array(CMilestones)
    GMilestones = np.array(GMilestones)
    TMilestones = np.array(TMilestones)
    
    #firstCol is a numpy array of the last index of each character in the first column
    firstCol = np.array([AMilestones[-1] - 1, AMilestones[-1] + CMilestones[-1] - 1, AMilestones[-1] + CMilestones[-1] + GMilestones[-1] - 1, AMilestones[-1] + CMilestones[-1] + GMilestones[-1] + TMilestones[-1] - 1])
    return AMilestones, CMilestones, GMilestones, TMilestones, firstCol


In [15]:
def rankHelper(index, character, flag = True):
    key = index // milestone
    if character == 'A':
        if flag:
            return AMilestones[key]
        else:
            return AMilestones[key] + reference[key * milestone + 1 : index + 1].count(character)
    elif character == 'C':
        if flag:
            return CMilestones[key]
        else:
            return CMilestones[key] + reference[key * milestone + 1 : index + 1].count(character)
    elif character == 'G':
        if flag:
            return GMilestones[key]
        else:
            return GMilestones[key] + reference[key * milestone + 1 : index + 1].count(character)
    elif character == 'T':
        if flag:
            return TMilestones[key]
        else:
            return TMilestones[key] + reference[key * milestone + 1 : index + 1].count(character)
    else:
        return 0

In [50]:
def rank(lastCol, start, end, character, milestone, milestones) -> Tuple[int, int]:
    #rank takes the reference sequence, start and end index, character and the milestone as input

    startModulus = start % milestone
    startFloor = start // milestone
    if startModulus == 0:
        firstLocation = milestones[startFloor] if (lastCol[start] == character) else (milestones[startFloor] + 1 if character in lastCol[start : end + 1] else -1)
    else:
        firstLocation = milestones[startFloor] + lastCol[startFloor * milestone + 1 : start + 1].count(character) if (lastCol[start] == character) else (milestones[startFloor] + lastCol[startFloor * milestone + 1 : start + 1].count(character) + 1 if character in lastCol[start : end + 1] else -1)

    endModulus = end % milestone
    endFloor = end // milestone
    if endModulus == 0:
        lastLocation = milestones[endFloor] if (lastCol[end] == character) else (milestones[endFloor] if character in lastCol[start : end + 1] else -1)
    else:
        lastLocation = milestones[endFloor] + lastCol[endFloor * milestone + 1 : end + 1].count(character) if (lastCol[end] == character) else (milestones[endFloor] + lastCol[endFloor * milestone + 1 : end + 1].count(character) if character in lastCol[start : end + 1] else -1)
    return firstLocation, lastLocation

In [51]:
def select(location, character, firstCol) -> int:
    # select takes the index i and character as input
    # Given index i, it returns the ith character in the first column of the BWT
    # The firstCol is an array which has the last index of each character in the first column
    # The firstCol array is created in the createMilestones function
    if character == 'A':
        return location - 1
    elif character == 'C':
        return firstCol[0] + location
    elif character == 'G':
        return firstCol[1] + location
    elif character == 'T':
        return firstCol[2] + location
    else:
        pass
    return None

In [10]:
def bandContraction(start, end, lastCol, letter, nextLetter, milestone, letterMilestones):
    # bandContraction takes the start and end of the band, last column, letter, milestone, letterMilestones and i as input
    #It performs rank query of letter on the last column to find the first and last occurences of letter
    #Once it has the indices of first and last occurences of letter, it performs select query on the first column in between these indices to find the indices of the first and last occurences of nextLetter
    #It then returns the indices of the first and last occurences of nextLetter as the new band
    
    return None

In [52]:
def BWTMapper(splitReads, firstCol, lastCol, mapping, milestone, AMilestones, CMilestones, GMilestones, TMilestones) -> Tuple[np.ndarray[int], int]:
    segment = -1
    locations = []
    for l in range(len(splitReads)):
        # Initialize the range of the read to be the entire last column
        start = 0
        end = len(lastCol) - 1
        # print(f'start = {start}, end = {end}, length of lastCol = {len(lastCol)}')
        for i in range(len(splitReads[l]) - 1, -1, -1):
            if splitReads[l][i] == 'A':
                firstLocation, lastLocation = rank(lastCol, start, end, 'A',  milestone, AMilestones)
                if firstLocation == -1 or lastLocation == -1:
                    break
                start, end = select(firstLocation, 'A', firstCol), select(lastLocation, 'A', firstCol)
            elif splitReads[l][i] == 'C':
                firstLocation, lastLocation = rank(lastCol, start, end, 'C',  milestone, CMilestones)
                if firstLocation == -1 or lastLocation == -1:
                    break                
                start, end = select(firstLocation, 'C', firstCol), select(lastLocation, 'C', firstCol)
            elif splitReads[l][i] == 'G':
                firstLocation, lastLocation = rank(lastCol, start, end, 'G',  milestone, GMilestones)
                if firstLocation == -1 or lastLocation == -1:
                    break
                start, end = select(firstLocation, 'G', firstCol), select(lastLocation, 'G', firstCol)
            elif splitReads[l][i] == 'T':
                firstLocation, lastLocation = rank(lastCol, start, end, 'T',  milestone, TMilestones)
                if firstLocation == -1 or lastLocation == -1:
                    break
                start, end = select(firstLocation, 'T', firstCol), select(lastLocation, 'T', firstCol)
            elif splitReads[l][i] == 'N':
                firstLocation, lastLocation = rank(lastCol, start, end, 'A',  milestone, AMilestones)
                if firstLocation == -1 or lastLocation == -1:
                    break
                start, end = select(firstLocation, 'A', firstCol), select(lastLocation, 'A', firstCol)
            else:
                pass
            if start > end:
                firstLocation = lastLocation = -1
                break
        # print(f"locations = {locations}, segment = {segment}, start = {start}, end = {end}, firstLocation = {firstLocation}, lastLocation = {lastLocation}, splitReads = {splitReads}, l = {l}")
        if firstLocation != -1 and lastLocation != -1:
            for i in range(start, end + 1):
                locations.append(mapping[i])
            segment = l
            break
    locations = np.array(locations)
    
    return locations, segment

In [53]:
def readSplitter(read) -> np.ndarray[str]:
    # readSplitter takes a read as input
    # It splits the read into 3 parts and returns a numpy array of three strings
    length = len(read)
    split = length // 3
    return np.array([read[:split], read[split:2*split], read[2*split:]])

In [54]:
def reverseComplement(read) -> str:
    # reverseComplement takes a read as input
    # A -> T, C -> G, G -> C, T -> A
    # It returns the reverse complement of the read
    return read.translate(str.maketrans('ACGT', 'TGCA'))[::-1]

In [55]:
def bruteForce(read, readLocation, reference) -> int:
    mismatches = 0
    for i in range(len(read)):
        #take care of the case where readLocation + i is greater than the length of the reference, meaning it wraps around to the start of the reference
        if readLocation + i < len(reference):
            if read[i] != reference[readLocation + i]:
                mismatches += 1
        else:
            if read[i] != reference[readLocation + i - len(reference)]:
                mismatches += 1
    return mismatches

In [170]:
l = []
l = np.array(l)
l
if l:
    print('j')

C:\Users\aadhi\AppData\Local\Temp\ipykernel_12740\1263013908.py:4: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if l:


In [56]:
def locationinReference(splitReads, locations, flag, reference):
    #Returns a list of tuples
    trueLocations = []
    if flag == 0:
        for location in locations:
            read = splitReads[0] + splitReads[1] + splitReads[2]
            readLocation = location
            mismatches = bruteForce(read, readLocation, reference)
            if mismatches <= 2:
                trueLocations.append((location, location + len(splitReads[0]) + len(splitReads[1]) + len(splitReads[2])))
    elif flag == 1:
        for location in locations:
            read = splitReads[0] + splitReads[1] + splitReads[2]
            readLocation = location - len(splitReads[0])
            mismatches = bruteForce(read, readLocation, reference)
            if mismatches <= 2:
                trueLocations.append((readLocation, location + len(splitReads[1]) + len(splitReads[2])))
    elif flag == 2:
        for location in locations:
            read = splitReads[0] + splitReads[1] + splitReads[2]
            readLocation = location - len(splitReads[0]) - len(splitReads[1])
            mismatches = bruteForce(read, readLocation, reference)
            if mismatches <= 2:
                trueLocations.append((readLocation, location + len(splitReads[2])))
    else:
        return None
    trueLocations = np.array(trueLocations)
    # print(f'mismatches = {mismatches}, trueLocations = {trueLocations}')
    return trueLocations

In [57]:
def removeN(read) -> str:
    return read.replace('N', 'A')

In [74]:
def forwardRead(read, reference, firstCol, lastCol, mapping, milestone, AMilestones, CMilestones, GMilestones, TMilestones) -> Optional[np.ndarray[Tuple[int, int]]]:
    #Some reads may have N, need to read these as A
    # print(f'length of lastCOl = {len(lastCol)}')
    read = removeN(read)
    splitReads = readSplitter(read)
    locations, flag = BWTMapper(splitReads=splitReads, firstCol=firstCol, lastCol=lastCol, mapping=mapping, milestone=milestone, AMilestones=AMilestones, CMilestones=CMilestones, GMilestones=GMilestones, TMilestones=TMilestones)
    if flag != -1:
        trueLocations = locationinReference(splitReads, locations, flag, reference)
        if trueLocations.any():
            return trueLocations
    return np.array([])

In [75]:
def backwardRead(read, reference, firstCol, lastCol, mapping, milestone, AMilestones, CMilestones, GMilestones, TMilestones) -> Optional[np.ndarray[Tuple[int, int]]]:
    read = removeN(read)
    read = reverseComplement(read)
    splitReads = readSplitter(read)
    locations, flag = BWTMapper(splitReads=splitReads, firstCol=firstCol, lastCol=lastCol, mapping=mapping, milestone=milestone, AMilestones=AMilestones, CMilestones=CMilestones, GMilestones=GMilestones, TMilestones=TMilestones)
    if flag != -1:
        trueLocations = locationinReference(splitReads, locations, flag, reference)
        if trueLocations.any():
            return trueLocations
    return np.array([])

In [60]:
def ExonMapper(redExon2, redExon3, redExon4, redExon5, greenExon2, greenExon3, greenExon4, greenExon5, locations) -> Tuple[int]:
    #checks if read is in the exon ambiguously or unambiguously
    #The ranges for the red exon are as follows:
    # 149256127 - 149256423
    # 149258412 - 149258580
    # 149260048 - 149260213
    # 149261768 - 149262007
    # The ranges for the green exon are as follows:
    # 149293258 - 149293554
    # 149295542 - 149295710
    # 149297178 - 149297343
    # 149298898 - 149299137
    red = green = -1
    for location in locations:
        start, end = location
        secondRed = 149256127 < start and end < 149256423
        thirdRed = 149258412 < start and end < 149258580
        fourthRed = 149260048 < start and end < 149260213
        fifthRed = 149261768 < start and end < 149262007
        secondGreen = 149293258 < start and end < 149293554
        thirdGreen = 149295542 < start and end < 149295710
        fourthGreen = 149297178 < start and end < 149297343
        fifthGreen = 149298898 < start and end < 149299137
        if secondRed:
            red = 2
        elif thirdRed:
            red = 3
        elif fourthRed:
            red = 4
        elif fifthRed:
            red = 5
        else:
            pass
        if secondGreen:
            green = 2
        elif thirdGreen:
            green = 3
        elif fourthGreen:
            green = 4
        elif fifthGreen:
            green = 5
        else:
            pass
    if red != -1 and green != -1:
        if red == 2:
            redExon2 += 0.5
            greenExon2 += 0.5
        elif red == 3:
            redExon3 += 0.5
            greenExon3 += 0.5
        elif red == 4:
            redExon4 += 0.5
            greenExon4 += 0.5
        elif red == 5:
            redExon5 += 0.5
            greenExon5 += 0.5
        else:
            pass
    elif red != -1:
        if red == 2:
            redExon2 += 1
        elif red == 3:
            redExon3 += 1
        elif red == 4:
            redExon4 += 1
        elif red == 5:
            redExon5 += 1
        else:
            pass
    elif green != -1:
        if green == 2:
            greenExon2 += 1
        elif green == 3:
            greenExon3 += 1
        elif green == 4:
            greenExon4 += 1
        elif green == 5:
            greenExon5 += 1
        else:
            pass
    else:
        pass
    return redExon2, redExon3, redExon4, redExon5, greenExon2, greenExon3, greenExon4, greenExon5

In [62]:
def colorBlindness(redExon2, redExon3, redExon4, redExon5, greenExon2, greenExon3, greenExon4, greenExon5) -> Tuple[float]:
    redFraction2 = redExon2 / (redExon2 + greenExon2)
    redFraction3 = redExon3 / (redExon3 + greenExon3)
    redFraction4 = redExon4 / (redExon4 + greenExon4)
    redFraction5 = redExon5 / (redExon5 + greenExon5)
    return redFraction2, redFraction3, redFraction4, redFraction5

In [17]:
reference = readFasta('data\\chrX.fa')
lastCol = readFastcol('chrX_last_col.txt')
reads = getReads('reads')
mapping = getMapping('chrX_map.txt')
milestone = 1000
# e = 0
AMilestones, CMilestones, GMilestones, TMilestones, firstCol = createMilestones(lastCol, milestone)
redExon2 = redExon3 = redExon4 = redExon5 = 0
greenExon2 = greenExon3 = greenExon4 = greenExon5 = 0

FileNotFoundError: chrX_last_col.txt not found.

In [105]:
def parallel(i):
    #forwardRead and backwardRead are run in parallel
    global redExon2, redExon3, redExon4, redExon5, greenExon2, greenExon3, greenExon4, greenExon5
    forwardLocations = forwardRead(reads[i], reference, firstCol, lastCol, mapping, milestone, AMilestones, CMilestones, GMilestones, TMilestones)
    backwardLocations = backwardRead(reads[i], reference, firstCol, lastCol, mapping, milestone, AMilestones, CMilestones, GMilestones, TMilestones)
    if forwardLocations.any():
        redExon2, redExon3, redExon4, redExon5, greenExon2, greenExon3, greenExon4, greenExon5 = ExonMapper(redExon2, redExon3, redExon4, redExon5, greenExon2, greenExon3, greenExon4, greenExon5, forwardLocations)
    if backwardLocations.any():
        redExon2, redExon3, redExon4, redExon5, greenExon2, greenExon3, greenExon4, greenExon5 = ExonMapper(redExon2, redExon3, redExon4, redExon5, greenExon2, greenExon3, greenExon4, greenExon5, backwardLocations)
    if i % 1000 == 0:
        print(i)

In [106]:
results = Parallel(n_jobs=8)(delayed(parallel)(i) for i in range(len(reads)))

KeyboardInterrupt: 

In [100]:
redFraction2, redFraction3, redFraction4, redFraction5 = colorBlindness(redExon2, redExon3, redExon4, redExon5, greenExon2, greenExon3, greenExon4, greenExon5)
print(redFraction2, redFraction3, redFraction4, redFraction5)

ZeroDivisionError: division by zero

In [98]:
def main():
    for i in range(100):
        parallel(i)
        # e += 1
        # if e % 1000 == 0:
        #     break
    redFraction2, redFraction3, redFraction4, redFraction5 = colorBlindness(redExon2, redExon3, redExon4, redExon5, greenExon2, greenExon3, greenExon4, greenExon5)
    return redFraction2, redFraction3, redFraction4, redFraction5
    

In [17]:
reference = readFasta('chrX.fa')
lastCol = readFastcol('chrX_last_col.txt')
# firstCol = getFirstcol(lastCol)
reads = getReads('reads')
mapping = getMapping('chrX_map.txt')
milestone = 1000
AMilestones, CMilestones, GMilestones, TMilestones, firstCol = createMilestones(lastCol, milestone)
bandIndexStart = 0
bandIndexEnd = len(lastCol)
redExon2 = redExon3 = redExon4 = redExon5 = 0
greenExon2 = greenExon3 = greenExon4 = greenExon5 = 0


In [225]:
print(firstCol)

[ 45648951  75462304 105328135 151100559]


In [26]:
forwardLocations = []

In [19]:
%pip install numba


  Obtaining dependency information for numba from https://files.pythonhosted.org/packages/01/01/8b7b670c77c5ea0e47e283d82332969bf672ab6410d0b2610cac5b7a3ded/numba-0.60.0-cp39-cp39-win_amd64.whl.metadata
  Obtaining dependency information for llvmlite<0.44,>=0.43.0dev0 from https://files.pythonhosted.org/packages/df/41/73cc26a2634b538cfe813f618c91e7e9960b8c163f8f0c94a2b0f008b9da/llvmlite-0.43.0-cp39-cp39-win_amd64.whl.metadata
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB 660.6 kB/s eta 0:00:05
    --------------------------------------- 0.0/2.7 MB 487.6 kB/s eta 0:00:06
   - -------------------------------------- 0.1/2.7 MB 930.9 kB/s eta 0:00:03
   --- ------------------------------------ 0.2/2.7 MB 1.2 MB/s eta 0:00:02
   ---- ----------------------------------- 0.3/2.7 MB 1.6 MB/s eta 0:00:02
   ------- -------------------------------- 0.5/2.7 MB 2.0 MB/s eta 0:00:02
   ----------- -----------------------

In [38]:
for i in range(669, len(reads), 1000):
    read = reads[i]
    temp = backwardRead(read, reference, firstCol, lastCol, mapping, milestone, AMilestones, CMilestones, GMilestones, TMilestones)
    read = removeN(read)
    read = reverseComplement(read)
    if temp is not None:
        for location in temp:
            mis = 0
            for j in range(len(read)):
                if reference[location[0] + j] != read[j]:
                    mis += 1
                if mis > 2:
                    print(f'index = {i}, location = {location}, mismatches = {mis}')
                    print('DANGERRRRRRRRRRRRRR')
                    break
            print(f"index = {i}, location = {location}, mismatches = {mis}")

index = 48669, location = [8023935 8024036], mismatches = 0
index = 48669, location = [7401688 7401789], mismatches = 2
index = 49669, location = [7400874 7400975], mismatches = 0
index = 49669, location = [8023123 8023224], mismatches = 1
index = 66669, location = [7728621 7728722], mismatches = 1
index = 66669, location = [6092475 6092576], mismatches = 1
index = 67669, location = [7728305 7728406], mismatches = 0
index = 67669, location = [6092159 6092260], mismatches = 0
index = 68669, location = [6091862 6091963], mismatches = 0
index = 75669, location = [7401519 7401620], mismatches = 0
index = 75669, location = [8023766 8023867], mismatches = 0
index = 77669, location = [7729284 7729385], mismatches = 1
index = 77669, location = [6093135 6093236], mismatches = 1
index = 79669, location = [6091922 6092023], mismatches = 1
index = 80669, location = [6091901 6092002], mismatches = 0
index = 373669, location = [23922269 23922370], mismatches = 2
index = 374669, location = [23921954 

In [32]:
print(forwardRead(reads[3669], reference, firstCol, lastCol, mapping, milestone, AMilestones, CMilestones, GMilestones, TMilestones))

None


In [279]:
mis = 0
ind = 0
for i in range(len(reads[ind])):
    if reference[2355249 + i] != reads[ind][i]:
        mis += 1
print(mis)

2


In [265]:
len(reads[9999])

101

In [274]:
print(reads[9999], reference[2483665:2483665 + len(reads[9999])])

TTGTCCCCGTAGCAACCGAGATCCCCAGTGCCTAGATCATCCGCCATGATCAGTAGGATATTTGGTTTAAAGGCATTTGCAGTTTTAGGTTCACACGTCTT TTGTTCCCGTAGCAACCGAGATCCCCAGTGCCTAGATCATCCGCCATGATCAGTAGGATATTTGGTTTAAAGGCATTTGCAGTTTTAGGTTCACACGTCTT


In [99]:
print(main())

0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
50
52
54
56
58
60
62
64
66
68
70
72
74
76
78
80
82
84
86
88
90
92
94
96
98
None


In [ ]:
#armaan 1
r2 = 271.5
r3 = 160
r4 = 227
r5 = 403
g2 = 288
g3 = 170
g4 = 141.5
g5 = 359

In [2]:
print(r2/(r2 + g2), r3/(r3 + g3), r4/(r4 + g4), r5/(r5 + g5))

0.48525469168900803 0.48484848484848486 0.6160108548168249 0.5288713910761155


In [3]:
#armaan 2
r2 = 208
r3 = 147.5
r4 = 240
r5 = 484
g2 = 128
g3 = 70.5
g4 = 54
g5 = 130

In [ ]:
print(r2/(r2 + g2), r3/(r3 + g3), r4/(r4 + g4), r5/(r5 + g5))

0.6190476190476191 0.676605504587156 0.8163265306122449 0.7882736156351792


In [5]:
#armaan 3
r2 = 144.5
r3 = 90
r4 = 173
r5 = 280.5
g2 = 191.5
g3 = 128
g4 = 121
g5 = 333.5

In [6]:
print(r2/(r2 + g2), r3/(r3 + g3), r4/(r4 + g4), r5/(r5 + g5))

0.43005952380952384 0.41284403669724773 0.5884353741496599 0.4568403908794788


In [7]:
#sai
r2 = 126.5
r3 = 34
r4 = 58
r5 = 176.5
g2 = 170.5
g3 = 69
g4 = 61
g5 = 229.5

In [8]:
print(r2/(r2 + g2), r3/(r3 + g3), r4/(r4 + g4), r5/(r5 + g5))

0.42592592592592593 0.3300970873786408 0.48739495798319327 0.43472906403940886


In [18]:
r2 = 148.5
r3 = 80.5
r4 = 180
r5 = 262.5
g2 = 181.5
g3 = 139.5
g4 = 130
g5 = 345.5

In [19]:
print(r2/(r2 + g2), r3/(r3 + g3), r4/(r4 + g4), r5/(r5 + g5))

0.45 0.3659090909090909 0.5806451612903226 0.4317434210526316
